In [1]:
import boto3
from botocore.client import Config
import os

In [2]:
# MinIO connection details
minio_endpoint = os.getenv('AWS_S3_ENDPOINT')
access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket_name = "images"
region = os.getenv('AWS_DEFAULT_REGION')

# Initialize the S3 client
s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)

# List objects in a specific bucket
objects = s3_client.list_objects_v2(Bucket=bucket_name)
if 'Contents' in objects:
    print("Objects found in bucket")
else:
    print("  No objects found.")

Objects found in bucket


In [3]:
folder_to_download = "validate/"
local_download_path = os.getcwd()
continuation_token = None

while True:
    if continuation_token:
        objects = s3_client.list_objects_v2(
            Bucket=bucket_name, Prefix=folder_to_download, ContinuationToken=continuation_token
        )
    else:
        objects = s3_client.list_objects_v2(
            Bucket=bucket_name, Prefix=folder_to_download
        )

    if 'Contents' in objects:
        for obj in objects['Contents']:
            object_key = obj['Key']

            # Only download if the object is in the specific folder
            if object_key.startswith(folder_to_download):
                local_file_path = os.path.join(local_download_path, object_key)

                os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
                s3_client.download_file(bucket_name, object_key, local_file_path)
                print(f"Downloaded {object_key} to {local_file_path}")

    if objects.get('IsTruncated'):
        continuation_token = objects['NextContinuationToken']
    else:
        break

Downloaded validate/Avulsion fracture/000002_png.rf.c3e00ebc2db78bc94e644c3f6605dad0.jpg to /opt/app-root/src/Beroepsproduct/Models/Scoring models/validate/Avulsion fracture/000002_png.rf.c3e00ebc2db78bc94e644c3f6605dad0.jpg
Downloaded validate/Avulsion fracture/1-s2-0-S0899707114002836-gr2_jpg.rf.bb8ed4dee892edbb760f6ce687d74ba5.jpg to /opt/app-root/src/Beroepsproduct/Models/Scoring models/validate/Avulsion fracture/1-s2-0-S0899707114002836-gr2_jpg.rf.bb8ed4dee892edbb760f6ce687d74ba5.jpg
Downloaded validate/Avulsion fracture/1b62e6fbfbc5a2f70c6af413189cfc82_jpg.rf.2761e933cc6d206308877cccaf0642b2.jpg to /opt/app-root/src/Beroepsproduct/Models/Scoring models/validate/Avulsion fracture/1b62e6fbfbc5a2f70c6af413189cfc82_jpg.rf.2761e933cc6d206308877cccaf0642b2.jpg
Downloaded validate/Comminuted fracture/images94_jpg.rf.1be75680f0289a4d2bcc8d0c112a3453.jpg to /opt/app-root/src/Beroepsproduct/Models/Scoring models/validate/Comminuted fracture/images94_jpg.rf.1be75680f0289a4d2bcc8d0c112a3453.

In [ ]:
import os
import zipfile

def zip_directory(folder_path, zip_filename):
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Example usage
folder_path = './validate'
zip_filename = './images_zipped.zip'

zip_directory(folder_path, zip_filename)